Read drive table from amazon.db

In [12]:
import sqlite3

import pandas as pd


db = sqlite3.connect('../data/amazon.db')

drive_df = pd.read_sql('select * from drive', db)
drive_df.head()

,asin,catagory,manufacturer,brand,model_name,product_name,drive_type,drive_subtype,capacity,last_update
0,B01K8A29BE,drive,ADATA USA,ADATA,su800,ADATA SU800 128GB 3D-NAND 2.5 Inch SATA III Hi...,ssd,sata,128gb,5722242
1,B01K8A2A0E,drive,ADATA USA,ADATA,su800,ADATA SU800 256GB 3D-NAND 2.5 Inch SATA III Hi...,ssd,sata,512gb,5722182
2,B01K8A29E6,drive,ADATA USA,ADATA,su800,ADATA USA Ultimate Su800 1TB 3D Nand 2.5 Inch ...,ssd,sata,1tb,5722216
3,B07GBRG2G8,drive,ADATA,ADATA,su800,ADATA SU800 2TB 3D-NAND 2.5 Inch SATA III High...,ssd,sata,2tb,5722242
4,B07TDW8Z3M,drive,ADATA,ADATA,su760,ADATA SU760 256GB 3D NAND 2.5 Inch SATA III In...,ssd,sata,256gb,5722242


Query all time series

In [13]:
def search(asin):
    drive_ts = pd.read_sql_query(f'select keepa_minute, value1 from drive_ts where asin = "{asin}" and keepa_idx=1 and value1 != -1', db)
    drive_ts['ts'] = pd.to_datetime((drive_ts['keepa_minute'] + 21564000) * 60, unit='s')
    drive_ts.set_index('ts', inplace=True)
    drive_ts = drive_ts.resample('1D').ffill()
    drive_ts = drive_ts.resample('M').mean()
    if '2021-01' not in drive_ts.index:
        return None
    base_price = drive_ts.loc['2021-01']['value1'][0]
    price_dict = {'base_price': base_price}
    for d in ['2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09', '2021-10']:
        if d in drive_ts.index:
            price_dict[d] = drive_ts.loc[d]['value1'][0] / base_price
        else:
            price_dict[d] = 1
    return price_dict

print(search(drive_df['asin'][1]))

{'base_price': 3647.3225806451615, '2021-01': 1.0, '2021-02': 1.0375223160982932, '2021-03': 1.0353772541944157, '2021-04': 1.0412295364695268, '2021-05': 1.113879381251824, '2021-06': 1.0717816280022758, '2021-07': 1.0475293410101973, '2021-08': 0.9649499854068826, '2021-09': 0.9587489423675049, '2021-10': 0.9327301511493186}


In [14]:
df_dict = []

for idx, row in drive_df.iterrows():
    ts = search(row['asin'])
    if ts is None:
        continue
    ts['asin'] = row['asin']
    ts['brand'] = row['brand']
    ts['product'] = row['model_name']
    ts['drive_type'] = row['drive_type']
    ts['drive_subtype'] = row['drive_subtype']
    ts['capacity'] = row['capacity']
    df_dict.append(ts)


df = pd.DataFrame(df_dict)
df.insert(0, 'CHIA_IMPACT', [0] * len(df))
len(df)

286

In [15]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, train_size=30)
df_train.to_csv('drive_train.csv', index=False)
df_test.to_csv('drive_test.csv', index=False)

Close database connection.

In [16]:
db.close()